# Possible differential operators for HCurlDiv:

For $H \in Hcd$ we can apply:
$$
\operatorname{div} H  \qquad \in H(d)^\ast
$$
and
$$
\operatorname{sym-curl} H^T \qquad \in H(cc)^\ast
$$


In [1]:
from ngsolve import *
mesh = Mesh(unit_cube.GenerateMesh(maxh=0.2))

importing NGSolve-6.2.2204


In [2]:
n = specialcf.normal(3)
Cn = CF( (0, n[2], -n[1], -n[2], 0, n[0], n[1], -n[0], 0) , dims=(3,3) )
Pn = OuterProduct(n,n)
Qn = Id(3) - Pn     

def CurlTHcc2HcdXXX(E,dH):
    return InnerProduct(curl(E).trans, dH)*dx \
       -InnerProduct( (E*Cn).trans, Qn*dH*Pn)*dx(element_boundary= True)

# shorter:
def CurlTHcc2Hcd(E,dH):
    return InnerProduct(curl(E).trans, dH)*dx \
       +InnerProduct(Cross(E*n, n), dH*n)*dx(element_boundary= True)

def DivHcdHd(H,dv):
    return div(H)*dv*dx - H*n*n * dv*n * dx(element_boundary= True)

In [3]:
order = 2
fescc = HCurlCurl(mesh, order=order)

fescd = HCurlDiv(mesh, order=order)
fesd = HDiv(mesh, order=order)
fescd_d = fescd*fesd

E, dE = fescc.TnT()
H, dH = fescd.TnT()
v, dv = fesd.TnT()
# (H,v), (dH, dv) = fescd_d.TnT()

In [4]:
massE = BilinearForm(InnerProduct(E,dE)*dx).Assemble()
massH = BilinearForm(InnerProduct(H,dH)*dx).Assemble()
massv = BilinearForm(InnerProduct(v,dv)*dx).Assemble()

bfcurlT = BilinearForm(CurlTHcc2Hcd(E, dH)).Assemble()
bfdiv = BilinearForm(DivHcdHd(H, dv)).Assemble()

In [5]:
massEinv = massE.mat.Inverse(inverse="sparsecholesky")
massHinv = massH.mat.Inverse(inverse="sparsecholesky")
massvinv = massv.mat.Inverse(inverse="sparsecholesky")

# $T \circ curl : H(cc) \rightarrow H(cd)$ 

In [6]:
gfE = GridFunction(fescc)
gfH = GridFunction(fescd)

gfE.Set ( ( (y,0,0), (0,0,0), (0,0,0)) )
gfH.vec.data = massHinv@bfcurlT.mat * gfE.vec

In [7]:
from ngsolve.webgui import Draw
Draw (gfE[0,0], mesh)
Draw (gfH[2,0], mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

# $sym \circ curl \circ T: H(cd) \rightarrow H(cc)$ 

In [8]:
gfH.Set ( ( (0,z,0), (0,0,0), (0,0,0)) )
gfE.vec.data = massEinv@bfcurlT.mat.T * gfH.vec

Draw (gfH[0,1], mesh)
Draw (gfE[0,0], mesh, min=-2, max=2, clipping = True)
Draw (gfE[1,1], mesh, min=-2, max=2, clipping = True);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

# $div : H(cd) \rightarrow H(d)$

In [9]:
gfH = GridFunction(fescd)
gfv = GridFunction(fesd)
gfH.Set ( ( (x*(1-x),0,0), (0,0,0), (0,0,0)) , dual = True)
gfv.vec.data = massvinv@bfdiv.mat * gfH.vec

In [10]:
Draw (gfv[0], mesh, min=-2, max=2, clipping = True);

Draw(gfH[0,0], mesh, min=-0.3, max=0.3, clipping = True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

# $\operatorname{dev} \operatorname{grad} : H(d) \rightarrow H(cd)$

In [11]:
gfv.Set ( (x,0,0))
gfH.vec.data = massHinv@bfdiv.mat.T * gfv.vec

In [12]:
Draw (gfH[0,0], mesh, min=-1, max=0, clipping = True);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…